<a href="https://colab.research.google.com/github/herrtunante/EMStrata/blob/main/PythonColabs/IPC_Data_Update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SET-UP THE LIBRARIES USED IN THE SCRIPTS


In [ ]:
!pip install requests
!pip install google-cloud-storage
!pip install pycountry
!pip install geojson


import ee
import requests
import pycountry
import datetime

from datetime import datetime
from geojson import loads

Define global variables

In [ ]:
# Base URL for the IPC API
base_url = "https://api.ipcinfo.org"
api_key = 'YOUR_API_KEY'  # Replace with your actual API key



# Define the prefix and asset folder
prefix = "users/YOUR_USER_NAME_IN_GEE"
assetFolder = f"{prefix}/IPC_updates"
assetId = f"{assetFolder}/IPC_phase3AndAbove"
assetId_temp = f"{assetId}_temp"
tempFolder = f"{assetFolder}/temporary"



# SET-UP GEE CONNECTION

In [ ]:
ee.Authenticate()

# Set your Earth Engine project ID
project_id = "YOUR_PROJECT_ID"

# Initialize Earth Engine with your project ID
ee.Initialize(project=project_id)


#Connect to API and download latest IPC data. Save to Asset folder on GEE

Define auxilliary methods that can be used in multiple blocks later


In [ ]:
import time
# Function to check task completion
def check_tasks_completion(task_ids):
    while True:
        all_done = True
        for task_id in task_ids:
            task = ee.data.getTaskStatus(task_id)[0]
            if task['state'] not in ['COMPLETED', 'FAILED', 'CANCELLED']:
                all_done = False
                break  # Exit the loop early if any task is not done
        if all_done:
            print("All tasks completed.")
            break
        else:
            print("Tasks still running. Waiting...")
            time.sleep(30)  # Wait for 30 seconds before checking again

# Function to check if the asset exists
def check_asset_exists(asset_name):
    try:
        asset_info = ee.data.getInfo(asset_name)
        return asset_info is not None
    except ee.EEException:
        return False

#Create folder if it does not exists
def create_folder_if_it_does_not_exist( folder ):
  if not ee.data.getInfo(folder):
      try:
          ee.data.createAsset({'type': 'Folder'}, folder)
          print(f"Folder '{folder}' created successfully.")
      except ee.EEException as e:
          print(f"Failed to create folder 1: {folder}")
          print(f"Failed to create folder 2: {e}")
  else:
      print(f"Folder '{folder}' already exists.")

def delete_asset_if_exists( asset ):
  if ( check_asset_exists(asset) is False ):
    print(f"Asset {asset} does not exist!")
  else :
    print(f"Deleteing asset : {asset}" )
    ee.data.deleteAsset(asset)



# Polling for asset rename completion
def waitForAssetToBeRenamed(new_asset_name):
  while not check_asset_exists(new_asset_name):
      print("Waiting for asset to be renamed...")
      time.sleep(10)  # Wait for 10 seconds before checking again


# Delete assets inside a folder
def delete_assets_in_folder(folder_path):
    # Get a list of assets in the folder
    asset_list = ee.data.getList({'id': folder_path})
    for asset in asset_list:
        asset_id = asset['id']
        # Delete the asset
        ee.data.deleteAsset(asset_id)
        print(f"Deleted asset: {asset_id}")


#Check if the folder exist in the GEE account

In [ ]:
# Create the folder if it doesn't exist
create_folder_if_it_does_not_exist( assetFolder )
create_folder_if_it_does_not_exist( tempFolder )


#Delete all content inside the folder before downloading new dataset

In [ ]:
# Delete assets in the 'temp' folder where the old country data is
delete_assets_in_folder(tempFolder)

#Finally, download all latest IPC data and save to the asset folder in the GEE

---

account

In [ ]:
# Function to convert month abbreviation to number
def month_to_number(month_str):
    return ee.Dictionary({
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
        'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
        'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    }).get(month_str)

# Function to parse date string into number and add parsed_date property to each feature
def add_parsed_date(feature):
    date_str = ee.String(feature.get("to"))
    parts = date_str.split(' ')
    month_num = month_to_number(parts.get(0))
    year = ee.Number.parse(parts.get(1))
    feature = feature.set("parsed_date", year.multiply(100).add(month_num))
    feature = feature.set("month", month_num)
    feature = feature.set("year", year)
    return feature

# Function to get newest country analysis
def get_newest_country_analysis(country_features):
    country_features_with_dates = country_features.filter(ee.Filter.stringContains("to", "20")) \
        .map(add_parsed_date)
    unique_areas = country_features_with_dates.distinct('title')
    newest_country_analysis = unique_areas.map(lambda feature:
        country_features_with_dates.filter(ee.Filter.eq('title', feature.get('title')))
        .sort('parsed_date', False).first())
    return newest_country_analysis

# Define the function to export to a temporary folder
def export_to_temporary_folder(feature_collection, country_code, year):
    # Modify the description to include the country code
    description = f"ExportIPCData_{country_code}_{year}"
    task = ee.batch.Export.table.toAsset(
        collection=feature_collection,
        description=description,
        assetId=f"{tempFolder}/{description}"  # Use the modified description for the assetId as well
    )
    # Start the export task and monitor its status
    print(f"Starting task to upload FeatureCollection for country code {country_code}..")
    task.start()
    return task.id

def get_features_for_country_in_year( iso_code, year ):
        # Construct the request URL
        request_url = f"{base_url}/areas?format=geojson&year={year}&type=A&country={iso_code}&key={api_key}"

        # Make the API request for the current year
        response_current_year = requests.get(request_url)
        if response_current_year.status_code == 200:
            geojson_data = response_current_year.json()
            if isinstance(geojson_data, dict) and geojson_data.get('type') == 'FeatureCollection' and 'features' in geojson_data and len(geojson_data['features']) > 0:
                return ee.FeatureCollection(geojson_data)
        return None


try:
    # Initialize variables
    listCountryData = ee.List([])
    sequence = 0
    task_id_array = []

    # Get current year and previous year
    current_year = datetime.now().year

    # Loop through all countries using their ISO CODE 2
    for country in pycountry.countries:
        iso_code = country.alpha_2

        # Loop backwards from 2024
        for year in range(current_year, 2022, -1):  # Adjust the end year as needed
            featureCollection = get_features_for_country_in_year( iso_code, year )
            # Break the loop if example_variable is not None
            if featureCollection is not None:
                print(f"Found data for {iso_code} in {year}")  # You can remove or replace this print statement as needed

                try:
                  task_id = export_to_temporary_folder(featureCollection, iso_code, year)
                  print(f"Task {task_id} started to run")
                  task_id_array.append(task_id)
                except ee.EEException as e:
                  print('Failed to upload the FeatureCollection! ', e)

                break

    # Wait until all export tasks are finished
    check_tasks_completion(task_id_array)

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")



**AT THIS POINT, WE HAVE THE IPC DATA DOWNLOADED INTO FEATURECOLLECTIONS IN THE TEMPFOLDER**

# **NOW WE NEED TO READ THOSE FEATURE COLLECTIONS, PUT THEM TOGETHER INTO A SINGLE FEATURE COLLECTION WITH ALL THE IPC LATEST DATA AND GENERATE A SINGLE IMAGE!**

#Functions to read the assets, selecting the latest data and flatten everything into one FeatureCollection (IPC>=3)

In [ ]:

# Function to convert month abbreviation to number
def month_to_number(month_str):
    return ee.Dictionary({
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
        'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
        'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    }).get(month_str)

# Function to parse date string into number and add parsed_date property to each feature
def add_parsed_date(feature):
    date_str = ee.String(feature.get("to"))
    parts = date_str.split(' ')
    month_num = month_to_number(parts.get(0))
    year = ee.Number.parse(parts.get(1))
    feature = feature.set("parsed_date", year.multiply(100).add(month_num))
    feature = feature.set("month", month_num)
    feature = feature.set("year", year)
    return feature

# Function to get newest country analysis
def get_newest_country_analysis(country_features):
    country_features_with_dates = country_features.filter(ee.Filter.stringContains("to", "20")) \
        .map(add_parsed_date)
    unique_areas = country_features_with_dates.distinct('title')
    newest_country_analysis = unique_areas.map(lambda feature:
        country_features_with_dates.filter(ee.Filter.eq('title', feature.get('title')))
        .sort('parsed_date', False).first())
    return newest_country_analysis

# Function to read country data from export folder
def read_country_data_from_export_folder( folder_path ):
    features_by_country = []
    assets_list = ee.data.listAssets(folder_path)
    for country_asset in assets_list['assets']:
        asset_id = country_asset['id']
        print(f"Reading asset: {asset_id}")
        collection_per_country = ee.FeatureCollection(asset_id)
        newest_collection_per_country = get_newest_country_analysis(collection_per_country)
        features_by_country.append(newest_collection_per_country)
    return features_by_country


print(f"Started reading assets in: {tempFolder}")
# Read the country features
country_features_with_newest_analysis = read_country_data_from_export_folder( tempFolder )
print(f"Finised reading assets in: {tempFolder}")
# Flatten the country features into a single Feature Collection
all_latest_analysis = ee.FeatureCollection(country_features_with_newest_analysis).flatten()
print(f"All features are stored in Feature Collection: all_latest_analysis")



**all_latest_analysis** CONTAINS AL OF THE LATEST IPC DATA !!!

Generate the image (filtering with IPC level > 3) and export it to an asset !

In [ ]:
# Function to get IPC image
def get_ipc_image(analysis):
    food_insecure = analysis.filter(ee.Filter.gte("overall_phase", 3))
    return food_insecure.reduceToImage(["parsed_date"], ee.Reducer.max())


# Define the region of interest (example) coversing the whole world
region = ee.Geometry.Rectangle([-180, -85, 180, 85], 'EPSG:4326', False)

ipcImage = get_ipc_image(all_latest_analysis)

# Add a 0/1 band if there was an IPC 3 or more report
ipcImage = ipcImage.addBands( ipcImage.gt(0).select( [0], [ "bin"]) )

current_time = datetime.now()
# Format the time as YYYY-MM-DD HH:MM
formatted_time = current_time.strftime('%Y-%m-%d %H:%M')

ipcImage = ipcImage.set("generatedOn", formatted_time )
ipcImage = ipcImage.set( "producedWithColab", "https://colab.research.google.com/drive/10sQ_Op-0Resnd8bq3MWevnrzp-lfaeVK")


# Make sure the temp asset is not present before update
delete_asset_if_exists( assetId_temp )

# Export the image
task = ee.batch.Export.image.toAsset(
        image=ipcImage,
        description=f"IPC_phase3_latest_to_temp_asset",
        assetId=assetId_temp,
        pyramidingPolicy={'.default': 'max'},
        scale=500,  # Adjust the scale as needed
        maxPixels=1e10,
        crs="EPSG:4326",
        region=region
    )
task.start()
# Monitor the export task
check_tasks_completion( [task.id]) # monitor the task using a single value array!


# // 4.1 Copy the temporary image into the final assetId and delete the temporary asset

In [ ]:
# When the task is done, copy asset to final destination and clean-up
ee.data.copyAsset(
    assetId_temp,
    assetId,
    True,  # Overwrite the original asset with the temporary one if it was already present
)
print( "Finally we have the image in its final asset reference ", assetId)


# Make the asset public
acl = ee.data.getAssetAcl(assetId)
acl['all_users_can_read'] = True
ee.data.setAssetAcl(assetId, acl)
print(f"Set {assetId} to be publicly accessible")

#Delete Temporary asset not needed any more
#ee.data.deleteAsset(assetId_temp)
print(f" Deleted { assetId_temp}")

# Delete assets in the 'temp' folder where the old country data is
delete_assets_in_folder(tempFolder)